Training

In [2]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pickle

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test_submition.csv")
test.cls = None

In [15]:
user_tokens = {}
bad_users = set()
downloaded_users = set()

In [16]:
def load_pickle(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [17]:
ls

1482080100_downloaded_users.pkl      test_submition.csv
1482080101_bad_users.pkl             train.csv
1482080101_user_tokens.pkl           user_tokens_10138.pkl
1482141731_bad_users.pkl             user_tokens_11138.pkl
1482141731_downloaded_users.pkl      user_tokens_1.pkl
1482141731_user_tokens.pkl           user_tokens_2000.pkl
1482143177_bad_users.pkl             user_tokens_3138.pkl
1482143177_downloaded_users.pkl      user_tokens_4138.pkl
1482143177_user_tokens.pkl           user_tokens_5138.pkl
1482143179_new_downloaded_users.pkl  user_tokens_6138.pkl
empty_test_users.pkl                 user_tokens_7138.pkl
Project. Data Preraring.ipynb        user_tokens_8138.pkl
Project. Training.ipynb              user_tokens_9138.pkl
test.csv


In [21]:
downloaded_users = load_pickle('1482143177_downloaded_users.pkl')
bad_users = load_pickle('1482080101_bad_users.pkl')


In [23]:
user_tokens = load_pickle('user_tokens_1.pkl')
user_tokens.update(load_pickle('user_tokens_2000.pkl'))
user_tokens.update(load_pickle('user_tokens_3138.pkl'))
user_tokens.update(load_pickle('user_tokens_4138.pkl'))
user_tokens.update(load_pickle('user_tokens_5138.pkl'))
user_tokens.update(load_pickle('user_tokens_6138.pkl'))
user_tokens.update(load_pickle('user_tokens_7138.pkl'))
user_tokens.update(load_pickle('user_tokens_8138.pkl'))
user_tokens.update(load_pickle('user_tokens_9138.pkl'))
user_tokens.update(load_pickle('user_tokens_10138.pkl'))
user_tokens.update(load_pickle('user_tokens_11138.pkl'))
user_tokens.update(load_pickle('1482143177_user_tokens.pkl'))

In [24]:
len(user_tokens), len(downloaded_users), len(bad_users)

(11809, 11947, 338)

In [27]:
train_tokens = {}
for uid in user_tokens.keys():
    if uid in train.uid.tolist():
        train_tokens[uid] = user_tokens[uid]

In [30]:
print len(train_tokens), len(train.uid.tolist())

 7061 7199


In [31]:
test_tokens = {}
for uid in user_tokens.keys():
    if uid in test.uid.tolist():
        test_tokens[uid] = user_tokens[uid]

In [32]:
print len(test_tokens), len(test.uid.tolist())

4748 4748


In [33]:
best_train = {}
for uid in train_tokens:
    if len(train_tokens[uid]) != 0:
        if uid not in bad_users:
            best_train[uid] = train_tokens[uid]

In [34]:
print len(best_train), len(train_tokens)

5394 7061


In [39]:
not_empty_test = {}
empty_test = {}
for uid in test_tokens:
    if len(test_tokens[uid]) != 0:
        not_empty_test[uid] = test_tokens[uid]
    else: 
        empty_test[uid] = test_tokens[uid]

In [45]:
len(test_tokens), len(not_empty_test), len(empty_test)

(4748, 3632, 1116)

In [36]:
def make_features(tokens):
    users = tokens.keys()
    vocabulary = [tokens[uid] for uid in users]
    v = DictVectorizer()
    return users, v.fit_transform(vocabulary)

In [48]:
def make_total_features(train_tokens, test_tokens):
    train_users = train_tokens.keys()
    vocabulary = [train_tokens[uid] for uid in train_users]
    test_users = test_tokens.keys()
    vocabulary.extend([test_tokens[uid] for uid in test_users])

    v = DictVectorizer()
    return train_users, test_users, v.fit_transform(vocabulary)

In [49]:
train_uids, test_uids, X = make_total_features(best_train, not_empty_test)

In [ ]:
len(train_uids), len(test_uids), X[:len(train

In [57]:
len(train_uids), len(test_uids), X[:len(train_uids)], X[len(train_uids):]

(5394, 3632, <5394x650877 sparse matrix of type '<type 'numpy.float64'>'
 	with 5138381 stored elements in Compressed Sparse Row format>, <3632x650877 sparse matrix of type '<type 'numpy.float64'>'
 	with 3010526 stored elements in Compressed Sparse Row format>)

In [54]:
Y = [train[train.uid == train_uids[i]].cls.values[0] for i in xrange(len(train_uids))]


In [55]:
len(Y)

5394

In [41]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)

In [42]:
classifier = svm.SVC()

In [58]:
Y_score = classifier.fit( X[:len(train_uids)], Y).decision_function(X[len(train_uids):])

In [59]:
len(Y_score)

3632

In [116]:
total_score = {}
for i in xrange(len(Y_score)):
    total_score[test_uids[i]] = Y_score[i]
for i, uid in enumerate(empty_test):
    total_score[uid] = Y_score[i]

In [117]:
count = 0
for i, uid in enumerate(total_score):
    if total_score[uid] == 0.79: count += 1
print count

0


In [118]:
submission = pd.DataFrame.from_dict(total_score, orient='index', dtype='float64')
submission.to_csv('project_submission.csv', sep=',', index=True, header=['cls'], index_label='uid')

In [215]:
len(Y_test), len(Y_score)

(1262, 1262)

In [134]:
!ls

1482080100_downloaded_users.pkl  test.csv	       user_tokens_3138.pkl
1482080101_bad_users.pkl	 test_submition.csv    user_tokens_4138.pkl
1482080101_user_tokens.pkl	 train.csv	       user_tokens_5138.pkl
Project. Data Preraring.ipynb	 user_tokens_1.pkl     user_tokens_6138.pkl
Project. Training.ipynb		 user_tokens_2000.pkl  user_tokens_7138.pkl


In [275]:

user_tokens.update(load_pickle('1482080101_user_tokens.pkl'))

In [7]:
len(user_tokens)

11809

In [8]:
len(test.uid.tolist())

4748

In [9]:
test_user_tokens = {}
empty_test_users = set()
for uid in test.uid.tolist():
    test_user_tokens[uid] = user_tokens[uid]
    if len(user_tokens[uid]) == 0:
        empty_test_users.add(uid)

In [10]:
len(test_user_tokens), len (empty_test_users)

(4748, 1116)

In [11]:
with open('empty_test_users.pkl', 'wb') as f:
        pickle.dump(empty_test_users, f, pickle.HIGHEST_PROTOCOL)
        print "pickle dumped", 'empty_test_users.pkl'

pickle dumped empty_test_users.pkl


In [270]:
!ls

1482080100_downloaded_users.pkl      test_submition.csv
1482080101_bad_users.pkl	     train.csv
1482080101_user_tokens.pkl	     user_tokens_10138.pkl
1482141731_bad_users.pkl	     user_tokens_11138.pkl
1482141731_downloaded_users.pkl      user_tokens_1.pkl
1482141731_user_tokens.pkl	     user_tokens_2000.pkl
1482143177_bad_users.pkl	     user_tokens_3138.pkl
1482143177_downloaded_users.pkl      user_tokens_4138.pkl
1482143177_user_tokens.pkl	     user_tokens_5138.pkl
1482143179_new_downloaded_users.pkl  user_tokens_6138.pkl
Project. Data Preraring.ipynb	     user_tokens_7138.pkl
Project. Training.ipynb		     user_tokens_8138.pkl
test.csv			     user_tokens_9138.pkl


In [12]:
len(user_tokens)

11809

In [13]:
not_empty = {}
for uid in user_tokens:
    if len(user_tokens[uid]) != 0:
        not_empty[uid] = user_tokens[uid]

In [14]:
len(not_empty)

9363

In [139]:
empty = {}
for uid in user_tokens:
    if len(user_tokens[uid]) == 0:
        empty[uid] = user_tokens[uid]

In [140]:
len(empty)

1176

In [148]:
empty.keys()

[713737670296514560,
 749083721462976513,
 749456311666941954,
 756571490456309763,
 729694776472313860,
 1014340771,
 2485624838,
 742711548104282113,
 1275559844,
 2811828234,
 193314827,
 154128397,
 1132672532,
 3312085007,
 424648720,
 751220875,
 1287741464,
 1955344410,
 3034107372,
 2798317598,
 3022688287,
 99136176,
 30851106,
 746270596628090880,
 3013675044,
 712122814690680838,
 149159975,
 1263110184,
 900909097,
 3012072137,
 2615568428,
 1611500893,
 748353880019378176,
 4805234739,
 25076062,
 767858743,
 2311772216,
 596529209,
 236633350,
 2436196411,
 2945368125,
 296493118,
 151771285,
 1340647489,
 745745537442381826,
 47987041,
 1181130824,
 3436343369,
 3234519210,
 218157131,
 1455702092,
 486029666,
 26685521,
 585273426,
 197558355,
 2411427924,
 2634041429,
 2944323673,
 2437498970,
 1933981711,
 133900381,
 32408612,
 3200026720,
 151318625,
 758025098649604096,
 231233414,
 941645928,
 3303083281,
 111177837,
 2413613166,
 19345520,
 245577064,
 35258484,


In [159]:
print float(train[train.uid.isin(empty.keys())].sum().values[1])/len(empty)
print float(train.cls.sum())/len(train)

0.750850340136
0.778997082928


In [266]:
Y_empty = train[train.uid.isin(empty.keys())].cls.tolist()
Y_test_empty = [0.79]*len(Y_empty)

In [267]:
len(Y_test), len(Y_score), len(Y_empty), len(Y_test_empty)

(1262, 1262, 1176, 1176)

In [268]:
Total_true = []
Total_true.extend(Y_test)
Total_true.extend(Y_empty)
Total_score = []
Total_score.extend(Y_score)
Total_score.extend(Y_test_empty)

In [269]:
fpr, tpr, thresholds = metrics.roc_curve(Total_true, Total_score)
print metrics.auc(fpr, tpr)

0.567712713039


In [86]:
from sklearn.feature_selection import SelectFdr 

In [87]:
SF = SelectFdr(alpha=0.02)

In [98]:
X_MNB_train = SF.fit_transform(X[:len(train_uids)], Y)

In [92]:
from sklearn.naive_bayes import MultinomialNB

In [95]:
X_MNB = X[len(train_uids):, SF.get_support()]

In [96]:
X_MNB

<3632x339 sparse matrix of type '<type 'numpy.float64'>'
	with 13850 stored elements in Compressed Sparse Row format>

In [93]:
MNB = MultinomialNB()

In [121]:

#Y_score_nb = MNB.fit(X_MNB_train, Y).predict_proba(X_MNB)
Y_score_nb = MNB.fit(X[:len(train_uids)], Y).predict_proba(X[len(train_uids):])

In [107]:
Y_score_nb

array([[ 0.10477903,  0.89522097],
       [ 0.07221989,  0.92778011],
       [ 0.04400425,  0.95599575],
       ..., 
       [ 0.87454845,  0.12545155],
       [ 0.10249515,  0.89750485],
       [ 0.03499494,  0.96500506]])

In [119]:
y_score_nb_answer = [a[1] for a in Y_score_nb]
y_score_nb_answer
total_score_nb = {}
for i in xrange(len(y_score_nb_answer)):
    total_score_nb[test_uids[i]] = y_score_nb_answer[i]
for i, uid in enumerate(empty_test):
    total_score_nb[uid] = y_score_nb_answer[i]
print len (total_score_nb)

4748


In [120]:
submission_nb = pd.DataFrame.from_dict(total_score_nb, orient='index', dtype='float64')
submission_nb.to_csv('project_submission_nb.csv', sep=',', index=True, header=['cls'], index_label='uid')

In [103]:
from sklearn.feature_selection import SelectFpr
from sklearn.naive_bayes import MultinomialNB



sfpr = SelectFpr(alpha = 0.02)
X1_train_fpr = sfpr.fit_transform(X[:len(train_uids)],Y)
bayes_clf = MultinomialNB()
#bayes_clf.fit(X1_train_fpr,Y)
#X1_test_fpr = sfpr.transform(X[len(train_uids):])
#bayes_clf.predict_proba(X1_test_fpr)

bayes_clf.fit(X1_train_fpr,Y)



array([[  2.03387400e-34,   1.00000000e+00],
       [  3.08475477e-16,   1.00000000e+00],
       [  2.09196770e-65,   1.00000000e+00],
       ..., 
       [  1.58868149e-12,   1.00000000e+00],
       [  8.83169278e-24,   1.00000000e+00],
       [  4.53051263e-14,   1.00000000e+00]])